In [1]:
import cv2, time, threading

caps = {
    "cam0": cv2.VideoCapture(3,cv2.CAP_V4L2),
    "cam1": cv2.VideoCapture(5,cv2.CAP_V4L2)
}

for c in caps.values():
    c.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
    c.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    c.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    c.set(cv2.CAP_PROP_FPS, 60)

last = time.time()
cnt = {"cam0":0, "cam1":0}
i = 0
while i<10:
    for name, cap in caps.items():
        ret, frame = cap.read()
        #cv2.imshow("testing",frame)
        if ret:
            cnt[name]+=1

    now = time.time()
    if now - last >= 1:
        print("FPS:", cnt)
        cnt = {"cam0":0, "cam1":0}
        i+=1
        last = now
for c in caps.values():
    c.release()
#cv2.destroyAllWindows()

FPS: {'cam0': 36, 'cam1': 36}
FPS: {'cam0': 60, 'cam1': 60}
FPS: {'cam0': 61, 'cam1': 61}
FPS: {'cam0': 60, 'cam1': 60}
FPS: {'cam0': 61, 'cam1': 61}
FPS: {'cam0': 61, 'cam1': 61}
FPS: {'cam0': 60, 'cam1': 60}
FPS: {'cam0': 61, 'cam1': 61}
FPS: {'cam0': 61, 'cam1': 61}
FPS: {'cam0': 61, 'cam1': 61}


In [7]:
import cv2, numpy as np, time, os, sys, threading, subprocess

# =============== CONFIG ===============
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11

CAM_SOURCES = {
    "Mobile":"http://192.168.137.110:8080/video",
    "Kreo1": 3,
    "Kreo2": 5
}

def create_detector():
    """Setup AprilTag detector with tuned parameters."""
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.cornerRefinementWinSize = 5
    params.cornerRefinementMaxIterations = 50
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 4.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

class CameraWorker(threading.Thread):
    def __init__(self, name, src, detector):
        super().__init__(daemon=True)
        self.name = name
        self.src = src
        self.cap = cv2.VideoCapture(src, cv2.CAP_V4L2)

        try:
            self.cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
            self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        except Exception:
            pass
        if isinstance(src, int):
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
            self.cap.set(cv2.CAP_PROP_FPS, 60)
        self.detector = detector
        self.lock = threading.Lock()
        self.latest_frame = None
        self.latest_ts = 0.0
        self.running = True
        self.opened = self.cap.isOpened()
        if not self.opened:
            print(f"[{self.name}] Error cannot open source {src}")

    def run(self):
        while self.running and self.opened:
            ret, frame = self.cap.read()
            if not ret:
                time.sleep(0.01)
                continue
            ts = time.time()
            with self.lock:
                self.latest_frame = frame
                self.latest_ts = ts
            time.sleep(0.001)

    def read_latest(self):
        with self.lock:
            if self.latest_frame is None:
                return None, 0.0
            return self.latest_frame.copy(), self.latest_ts
    
    def stop(self):
        self.running = False
        try: self.cap.release()
        except: pass


def get_camera_selection():
    print("\n=== Multi-Camera Live View Setup ===")
    print("Select cameras to open (comma separated):")
    print("1. Kreo Webcam #1")
    print("2. Kreo Webcam #2")
    print("3. Mobile IP Webcam")
    print("Example: 1,2 or 1,3 or 1,2,3")
    user_in = input("Cameras to open: ").strip()
    choices = [x.strip() for x in user_in.split(",") if x.strip()]
    selected = []
    for c in choices:
        if c == "1":
            selected.append(("Kreo1", CAM_SOURCES["Kreo1"]))
        elif c == "2":
            selected.append(("Kreo2", CAM_SOURCES["Kreo2"]))
        elif c == "3":
            selected.append(("Mobile", CAM_SOURCES["Mobile"]))
        else:
            print(f"[WARN] Ignoring invalid entry: {c}")
    if not selected:
        print("[ERROR] No valid cameras selected. Exiting.")
        sys.exit(1)
    return selected

if __name__ == "__main__":
    selected = get_camera_selection()

    workers = {}
    for name, src in selected:
        w = CameraWorker(name, src, create_detector())
        w.start()
        workers[name] = w
        time.sleep(0.05)
    
    # After tuning show live preview with settings applied
    print("[INFO] Preview without fine tuning. Press ESC to exit preview windows.")
    last_ts = {name: 0.0 for name in workers}
    fps_counter = {name: 0 for name in workers}
    fps = {name: 0.0 for name in workers}
    last_fps_update = time.time()

    try:
        while True:
            now = time.time()
            timestamps = {}

            # --- Collect frames from all cameras ---
            for name, w in workers.items():
                frame, ts = w.read_latest()
                if frame is None:
                    continue
                timestamps[name] = ts

                # --- FPS update ---
                if ts != last_ts[name]:
                    fps_counter[name] += 1
                    last_ts[name] = ts

                if now - last_fps_update >= 1.0:
                    fps[name] = fps_counter[name]
                    fps_counter[name] = 0

            if now - last_fps_update >= 1.0:
                last_fps_update = now

            # --- Drift calculation ---
            if len(timestamps) > 1:
                tvals = np.array(list(timestamps.values()))
                drift_ms = (tvals.max() - tvals.min()) * 1000.0
            else:
                drift_ms = 0.0

            # --- Draw every camera independently ---
            for name, w in workers.items():
                frame, ts = w.read_latest()
                if frame is None:
                    continue
                # AprilTag overlay
                corners, ids, _ = w.detector.detectMarkers(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
                if ids is not None and len(ids) > 0:
                    cv2.aruco.drawDetectedMarkers(frame, corners, ids)

                # resolution text
                h, wid = frame.shape[:2]
                cv2.putText(frame, f"{name} {wid}x{h}", (10, 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0), 2)

                cv2.putText(frame, f"FPS: {fps[name]:.0f}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0), 2)

                cv2.putText(frame, f"Drift: {drift_ms:.1f} ms", (10, 75),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

                cv2.imshow(f"Tuned - {name}", frame)

            # Exit on ESC
            if cv2.waitKey(1) & 0xFF == 27:
                break

            time.sleep(0.001)

    except KeyboardInterrupt:
        pass

    finally:
        for w in workers.values():
            w.stop()
        cv2.destroyAllWindows()




=== Multi-Camera Live View Setup ===
Select cameras to open (comma separated):
1. Kreo Webcam #1
2. Kreo Webcam #2
3. Mobile IP Webcam
Example: 1,2 or 1,3 or 1,2,3
[INFO] Preview without fine tuning. Press ESC to exit preview windows.


In [3]:
import cv2, time, threading, subprocess, json, os, numpy as np, sys

# =============== CONFIG ===============
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11

CAM_SOURCES = {
    "Mobile":"http://192.168.137.110:8080/video",
    "Kreo1": 0,
    "Kreo2": 4
}

# parameter grids. Tweak if needed
EXPOSURES = [80, 120, 160, 200, 240, 300]
FOCUSES = [80, 120, 160, 200, 240, 280, 320, 360]
GAINS = [0, 6, 12, 18]
BRIGHTNESSES = [0, 8, 16, 24]

# evaluation parameters
EVAL_SECONDS = 1
SAMPLE_SLEEP = 0.02

OUTPUT_DIR = "./camera_tune_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# AprilTag detector setup
def create_detector():
    """Setup AprilTag detector with tuned parameters."""
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.cornerRefinementWinSize = 5
    params.cornerRefinementMaxIterations = 50
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 4.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

class CameraWorker(threading.Thread):
    def __init__(self, name, src, detector):
        super().__init__(daemon=True)
        self.name = name
        self.src = src
        self.cap = cv2.VideoCapture(src)

        try:
            self.cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
            self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        except Exception:
            pass
        if isinstance(src, int):
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
            self.cap.set(cv2.CAP_PROP_FPS, 60)
        self.detector = detector
        self.lock = threading.Lock()
        self.latest_frame = None
        self.latest_ts = 0.0
        self.running = True
        self.opened = self.cap.isOpened()
        if not self.opened:
            print(f"[{self.name}] Error cannot open source {src}")

    def run(self):
        while self.running and self.opened:
            ret, frame = self.cap.read()
            if not ret:
                time.sleep(0.01)
                continue
            ts = time.time()
            with self.lock:
                self.latest_frame = frame
                self.latest_ts = ts
            time.sleep(0.001)

    def read_latest(self):
        with self.lock:
            if self.latest_frame is None:
                return None, 0.0
            return self.latest_frame.copy(), self.latest_ts
    
    def stop(self):
        self.running = False
        try: self.cap.release()
        except: pass

# v4l2 control helper
def v4l2_set(dev_idx, control, value):
    dev = f"/dev/video{dev_idx}"
    cmd = ["v4l2-ctl", "-d", dev, "-c", f"{control}={value}"]
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except subprocess.CalledProcessError:
        return False

def v4l2_get(dev_idx, control):
    dev = f"/dev/video{dev_idx}"
    cmd = ["v4l2-ctl", "-d", dev, "--get-ctrl", control]
    try:
        out = subprocess.check_output(cmd, stderr=subprocess.DEVNULL).decode().strip()
        return out
    except subprocess.CalledProcessError:
        return None
    
def evaluate_setting(worker, sample_duration=EVAL_SECONDS):
    start = time.time()
    end = start + sample_duration
    tag_counts = []
    frames = 0
    t0 = time.time()
    while time.time()<end:
        frame, ts = worker.read_latest()
        if frame is None:
            time.sleep(SAMPLE_SLEEP)
            continue
        frames += 1
        corners, ids, _ = worker.detector.detectMarkers(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        n = 0 if ids is None else len(ids)
        tag_counts.append(n)
        time.sleep(SAMPLE_SLEEP)
    elapsed = time.time()- t0
    avg_tags = float(np.mean(tag_counts)) if tag_counts else 0.0
    std_tags = float(np.std(tag_counts)) if tag_counts else 0.0
    fps = frames / elapsed if elapsed>0 else 0.0
    return { "avg_tags": avg_tags, "std_tags": std_tags, "fps": fps, "samples": len(tag_counts)}

def tune_camera(dev_idx, worker, brief_name):
    if not isinstance(worker.src, int):
        print(f"[{brief_name}] Skipping tuning for non-local source {worker.src}")
        return None
    print(f"[{brief_name}] Starting auto-tune (this may take a few minutes)...")

    try: 
        v4l2_set(dev_idx, "focus_automatic_continuous",0)
        v4l2_set(dev_idx, "auto_exposure", 1)
        v4l2_set(dev_idx, "exposure_dynamic_framerate",0)
    except Exception:
        pass

    best = {"score": -1.0}
    total_iters = len(EXPOSURES)*len(FOCUSES)*len(GAINS)*len(BRIGHTNESSES)
    it = 0
    for exp in EXPOSURES:
        ok = v4l2_set(dev_idx, "exposure_time_absolute", int(exp))
        if not ok:
            pass
        time.sleep(0.08)
        for f in FOCUSES:
            v4l2_set(dev_idx, "focus_absolute", int(f))
            time.sleep(0.04)
            for g in GAINS:
                v4l2_set(dev_idx, "gain", int(g))
                time.sleep(0.02)
                for b in BRIGHTNESSES:
                    v4l2_set(dev_idx, "brightness", int(b))
                    time.sleep(0.06)  # let camera settle a little
                    it += 1
                    print(f"[{brief_name}] Test {it}/{total_iters} exp={exp} focus={f} gain={g} bright={b} ...", end="\r")
                    stats = evaluate_setting(worker)
                    # scoring: primary = average tags detected, secondary = fps, penalize variance
                    score = stats["avg_tags"] * 100.0 + stats["fps"] * 0.2 - stats["std_tags"]*10.0
                    if score > best.get("score", -1.0):
                        best = {
                            "score": score,
                            "exp": int(exp),
                            "focus": int(f),
                            "gain": int(g),
                            "brightness": int(b),
                            "stats": stats.copy()
                        }
    print("")  # newline after progress

    if best.get("score", -1.0) < 0:
        print(f"[{brief_name}] No valid tuning results found.")
        return None

    print(f"[{brief_name}] Best found: exposure={best['exp']} focus={best['focus']} gain={best['gain']} brightness={best['brightness']}")
    print(f"[{brief_name}] stats: avg_tags={best['stats']['avg_tags']:.2f}, fps={best['stats']['fps']:.1f}, std={best['stats']['std_tags']:.2f}")

    # Apply the best settings finally
    v4l2_set(dev_idx, "exposure_time_absolute", int(best['exp']))
    v4l2_set(dev_idx, "focus_absolute", int(best['focus']))
    v4l2_set(dev_idx, "gain", int(best['gain']))
    v4l2_set(dev_idx, "brightness", int(best['brightness']))

    # Save to JSON
    out_file = os.path.join(OUTPUT_DIR, f"best_camera_settings_{brief_name}.json")
    with open(out_file, "w") as f:
        json.dump(best, f, indent=2)
    print(f"[{brief_name}] Best settings saved -> {out_file}")
    return best

def get_camera_selection():
    print("\n=== Multi-Camera Live View Setup ===")
    print("Select cameras to open (comma separated):")
    print("1. Kreo Webcam #1")
    print("2. Kreo Webcam #2")
    print("3. Mobile IP Webcam")
    print("Example: 1,2 or 1,3 or 1,2,3")
    user_in = input("Cameras to open: ").strip()
    choices = [x.strip() for x in user_in.split(",") if x.strip()]
    selected = []
    for c in choices:
        if c == "1":
            selected.append(("Kreo1", CAM_SOURCES["Kreo1"]))
        elif c == "2":
            selected.append(("Kreo2", CAM_SOURCES["Kreo2"]))
        elif c == "3":
            selected.append(("Mobile", CAM_SOURCES["Mobile"]))
        else:
            print(f"[WARN] Ignoring invalid entry: {c}")
    if not selected:
        print("[ERROR] No valid cameras selected. Exiting.")
        sys.exit(1)
    return selected

# ==================== MAIN ====================
if __name__ == "__main__":
    selected = get_camera_selection()

    workers = {}
    for name, src in selected:
        w = CameraWorker(name, src, create_detector())
        w.start()
        workers[name] = w
        time.sleep(0.05)

    # small warmup
    print("[INFO] Warmup for 1.2 seconds to let cameras settle...")
    time.sleep(1.2)
    
    for name, w in list(workers.items()):
        if isinstance(w.src,int):
            try:
                dev_idx = int(w.src)
            except Exception:
                print(f"[{name}] Invalid device index for tuning: {w.src}")
                continue
            best = tune_camera(dev_idx, w, name)
        else:
            print(f"[{name}] Not a local device; skipping tuning.")

    # After tuning show live preview with settings applied
    print("[INFO] Tuning complete. Press ESC to exit preview windows.")
    last_ts = {name: 0.0 for name in workers}
    fps_counter = {name: 0 for name in workers}
    fps = {name: 0.0 for name in workers}
    last_fps_update = time.time()

    try:
        while True:
            now = time.time()
            timestamps = {}

            # --- Collect frames from all cameras ---
            for name, w in workers.items():
                frame, ts = w.read_latest()
                if frame is None:
                    continue
                timestamps[name] = ts

                # --- FPS update ---
                if ts != last_ts[name]:
                    fps_counter[name] += 1
                    last_ts[name] = ts

                if now - last_fps_update >= 1.0:
                    fps[name] = fps_counter[name]
                    fps_counter[name] = 0

            if now - last_fps_update >= 1.0:
                last_fps_update = now

            # --- Drift calculation ---
            if len(timestamps) > 1:
                tvals = np.array(list(timestamps.values()))
                drift_ms = (tvals.max() - tvals.min()) * 1000.0
            else:
                drift_ms = 0.0

            # --- Draw every camera independently ---
            for name, w in workers.items():
                frame, ts = w.read_latest()
                if frame is None:
                    continue
                # AprilTag overlay
                corners, ids, _ = w.detector.detectMarkers(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
                if ids is not None and len(ids) > 0:
                    cv2.aruco.drawDetectedMarkers(frame, corners, ids)

                # resolution text
                h, wid = frame.shape[:2]
                cv2.putText(frame, f"{name} {wid}x{h}", (10, 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0), 2)

                cv2.putText(frame, f"FPS: {fps[name]:.0f}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0), 2)

                cv2.putText(frame, f"Drift: {drift_ms:.1f} ms", (10, 75),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

                cv2.imshow(f"Tuned - {name}", frame)

            # Exit on ESC
            if cv2.waitKey(1) & 0xFF == 27:
                break

            time.sleep(0.001)

    except KeyboardInterrupt:
        pass

    finally:
        for w in workers.values():
            w.stop()
        cv2.destroyAllWindows()



=== Multi-Camera Live View Setup ===
Select cameras to open (comma separated):
1. Kreo Webcam #1
2. Kreo Webcam #2
3. Mobile IP Webcam
Example: 1,2 or 1,3 or 1,2,3


[ WARN:0@460.112] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@460.208] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
[ WARN:0@460.259] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@460.260] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range


[Kreo1] Error cannot open source 0
[Kreo2] Error cannot open source 4
[INFO] Warmup for 1.2 seconds to let cameras settle...
[Kreo1] Starting auto-tune (this may take a few minutes)...
[Kreo1] Test 768/768 exp=300 focus=360 gain=18 bright=24 ...
[Kreo1] Best found: exposure=80 focus=80 gain=0 brightness=0
[Kreo1] stats: avg_tags=0.00, fps=0.0, std=0.00
[Kreo1] Best settings saved -> ./camera_tune_results/best_camera_settings_Kreo1.json
[Kreo2] Starting auto-tune (this may take a few minutes)...


KeyboardInterrupt: 